In [ ]:
!pip install pandas numpy tqdm matplotlib seaborn scikit-learn lz4


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import json
import lz4.frame
import pandas as pd
from collections import defaultdict
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from pathlib import Path

## Load Checkpoint ID → Name Mapping

In [22]:
def load_check_ids():
    all_ids = {}
    files = [
        "parameters/mail/dmarc-checkpoints.json",
        "parameters/mail/spf-checkpoints.json",
        "parameters/mail/server-checkpoints.json"
    ]
    
    for file in files:
        with open(file) as f:
            parsed = json.load(f)
            for section in parsed.values():
                for name, id_ in section.items():
                    all_ids[id_] = name  # ✅ id → name
    return all_ids

In [23]:
print(f"Loaded {len(CHECK_ID_TO_NAME)} checkpoint IDs.")
# Show a few examples
for i, (k, v) in enumerate(CHECK_ID_TO_NAME.items()):
    print(f"{k} → {v}")
    if i >= 4: break

Loaded 40 checkpoint IDs.
mail-49qbd59hbw8rx9d3p8f9fjfk8k → DMARC_RECORD_PRESENT
mail-51p7x6zqwr8xc9q9rs6j58ynes → POLICY_IS_NOT_NONE
mail-3pena79a6984qa07rvamvmx1t3 → SUBDOMAIN_POLICY_IS_NOT_NONE
mail-2edjx24zym9wdat2k26b2yv66a → POLICY_IS_PRESENT
mail-6txsgv29t387vvev78kh640x79 → STRICT_MODE


In [ ]:
def load_json_from_lz4(file_path):
    try:
        with lz4.frame.open(file_path, mode="rt") as f:
            data = json.load(f)

        reference = data.get("reference", "❓ No reference")
        timestamp = data.get("timestamp", "❓ No timestamp")
        print(f"Loaded JSON from: {file_path}")
        print(f"Reference: {reference}")
        print(f"Timestamp: {timestamp}")
        return data

    except Exception as e:
        print(f"❌ Error loading file: {e}")
        return None


In [ ]:
# Step 1: Load references from param files
PARAM_FILES = [
    "parameters/mail/dmarc-checkpoints.json",
    "parameters/mail/spf-checkpoints.json",
    "parameters/mail/server-checkpoints.json"
]

def load_all_references(param_paths):
    references = set()
    for path in param_paths:
        with open(path, "r") as f:
            content = json.load(f)
            for section in content.values():
                if isinstance(section, dict):
                    references.update(section.values())
    return sorted(references)

checkpoint_refs = load_all_references(PARAM_FILES)
CRITICITIES = ["OK", "INFO", "LOW", "MEDIUM", "HIGH"]
all_pairs = sorted([f"{ref}_{crit}" for ref in checkpoint_refs for crit in CRITICITIES])

# Step 2: Prepare file reading
BASE_DIR = Path("organized_by_company")
lz4_files = sorted(BASE_DIR.glob("**/*.json.lz4"))

rows = []

for file_path in lz4_files:
    try:
        with lz4.frame.open(file_path, mode="rt") as f:
            data = json.load(f)
    except Exception as e:
        print(f"❌ Failed to read {file_path.name}: {e}")
        continue

    company_ref = data.get("reference", "unknown")
    timestamp = data.get("timestamp", "unknown")
    score_data = data.get("mail", {}).get("score", {})
    score_value = score_data.get("value")
    internal_value = score_data.get("internal_value")

    row = {pair: 0 for pair in all_pairs}
    row["company_reference"] = company_ref
    row["timestamp"] = timestamp
    row["score_value"] = score_value
    row["internal_value"] = internal_value

    asset_checks = data.get("mail", {}).get("asset_checks", [])
    for asset in asset_checks:
        for cp in asset.get("checkpoints", []):
            ref = cp.get("reference")
            result = cp.get("result")
            ref = cp.get("reference", "").strip()
            if ref in checkpoint_refs and result in CRITICITIES:
                key = f"{ref}_{result}"
                row[key] += 1
    rows.append(row)

# Step 3: Build DataFrame and save
df = pd.DataFrame(rows)
csv_path = "mail_training_matrix_final.csv"
df.to_csv(csv_path, index=False)

print(f"✅ CSV with all pairs saved to: {csv_path}")


✔️ Ref = mail-49qbd59hbw8rx9d3p8f9fjfk8k | Result = HIGH
✔️ Ref = mail-1e9vyse1pv9tsvhajq08tk9cx2 | Result = LOW
✔️ Ref = mail-427ksdsnx28m6sj599te45f242 | Result = OK
✔️ Ref = mail-4rqj80t00989cbpk8t5nnrazqp | Result = OK
✔️ Ref = mail-f1jjp98c79s7a1fg5r3gf6j9b | Result = OK
✔️ Ref = mail-46cvtwg4ty9hha7p5wmg5ze7hq | Result = OK
✔️ Ref = mail-4hqpmkf90398n8ee4hk3tjfpes | Result = OK
✔️ Ref = mail-5v5dtcb48c9ffv8p5ygmg54nnx | Result = OK
✔️ Ref = mail-6c2t4ktp4488h8jhefxsamzxb0 | Result = OK
✔️ Ref = mail-awc908ksz9tybbe45bxffbnh4 | Result = OK
✔️ Ref = mail-5e0mwk5veh8j8ad1ek0eesvwqf | Result = OK
✔️ Ref = mail-3v9bn2thf88bdr1jq3h06yks8b | Result = OK
✔️ Ref = mail-4xgr1qyz418r9rrfxh4sxxpj5y | Result = OK
✔️ Ref = mail-6qk07qd2zf80aa8p75c25yrzkd | Result = OK
✔️ Ref = mail-3v9bn2thf88bdr1jq3h06yks8b | Result = OK
✔️ Ref = mail-4xgr1qyz418r9rrfxh4sxxpj5y | Result = OK
✔️ Ref = mail-6qk07qd2zf80aa8p75c25yrzkd | Result = OK
✔️ Ref = mail-3v9bn2thf88bdr1jq3h06yks8b | Result = OK
✔️ Ref = 

In [3]:
import pandas as pd

# Load your dataset
file_path = "data/mail_training_matrix_final.csv"
df = pd.read_csv(file_path)

# Drop duplicates across all feature columns (ignore timestamp + company_reference)
df_nodup = df.drop(columns=["timestamp", "company_reference"]).drop_duplicates()

# Merge back company_reference and timestamp (optional: keep the first occurrence)
df_final = df.loc[df_nodup.index]

# Print results
print("Original number of rows:", len(df))
print("Number of rows after removing duplicates:", len(df_final))

# Save cleaned version if needed
df_final.to_csv("mail_training_matrix_final_nodup.csv", index=False)


Original number of rows: 172426
Number of rows after removing duplicates: 2200


In [6]:
import pandas as pd

# Load dataset
file_path = "mail_training_matrix_final_nodup.csv"
df = pd.read_csv(file_path)

# Exclude metadata and target columns from the check
exclude_cols = ["company_reference", "timestamp", "score_value", "internal_value"]

# Total number of columns before cleaning
print("Total number of columns before cleaning:", len(df.columns))

# Find all-zero columns
all_zero_cols = [col for col in df.columns if col not in exclude_cols and (df[col] == 0).all()]

print("Number of all-zero columns:", len(all_zero_cols))
print("Dropping the following columns:")
for col in all_zero_cols:
    print(col)

# Drop all-zero columns
df_cleaned = df.drop(columns=all_zero_cols)

# Print new shape
print("New shape after dropping:", df_cleaned.shape)

# Save cleaned dataset
df_cleaned.to_csv("mail_training_matrix_final_clean.csv", index=False)


Total number of columns before cleaning: 204
Number of all-zero columns: 148
Dropping the following columns:
mail-10bm86dyrn89tbb5d895zwyt3e_HIGH
mail-10bm86dyrn89tbb5d895zwyt3e_LOW
mail-10bm86dyrn89tbb5d895zwyt3e_MEDIUM
mail-10bm86dyrn89tbb5d895zwyt3e_OK
mail-13sq4pyynh8k3btd7q1r5c5hfm_HIGH
mail-13sq4pyynh8k3btd7q1r5c5hfm_INFO
mail-13sq4pyynh8k3btd7q1r5c5hfm_LOW
mail-13sq4pyynh8k3btd7q1r5c5hfm_MEDIUM
mail-13sq4pyynh8k3btd7q1r5c5hfm_OK
mail-19mvyeegbc9m8rgv6hjnjfd4ha_INFO
mail-19mvyeegbc9m8rgv6hjnjfd4ha_LOW
mail-19mvyeegbc9m8rgv6hjnjfd4ha_MEDIUM
mail-19mvyeegbc9m8rgv6hjnjfd4ha_OK
mail-1e9vyse1pv9tsvhajq08tk9cx2_HIGH
mail-1e9vyse1pv9tsvhajq08tk9cx2_INFO
mail-1e9vyse1pv9tsvhajq08tk9cx2_MEDIUM
mail-24cwrkk7yd9e8v8ak7pp09wvpd_HIGH
mail-24cwrkk7yd9e8v8ak7pp09wvpd_INFO
mail-24cwrkk7yd9e8v8ak7pp09wvpd_LOW
mail-24cwrkk7yd9e8v8ak7pp09wvpd_MEDIUM
mail-274e1wpk469dh92xgg9x24zady_HIGH
mail-274e1wpk469dh92xgg9x24zady_LOW
mail-274e1wpk469dh92xgg9x24zady_MEDIUM
mail-274e1wpk469dh92xgg9x24zady_OK
mail